# Mapbox speed estimates

Through the World Bank Group's [Development Data Partnership](https://datapartnership.org/) we can access [MapBox's traffic estimates](https://docs.mapbox.com/data/traffic/guides/). Thes data provide users with access to both **live** and **typical** speeds along OSM segments.

This notebook will examine how to attach the speed data to OSM road segments and integrate those into a GOSTnets speed analysis.

In [23]:
import sys, os
import rasterio
import openlr
import osmnx

import pandas as pd
import geopandas as gpd

from shapely.geometry import Point, LineString


In [41]:
# Define input data
input_speed_file = r"C:\Users\WB411133\Downloads\MapBox\0331123\Africa-Casablanca\0331123-Africa-Casablanca.csv\0331123-Africa-Casablanca.csv"


# Define working data
base_folder = os.path.join(r"C:\WBG\Work\Projects\Congestion/")
roads_data = os.path.join(base_folder, "roads")

for folder in [base_folder, roads_data]:
    if not os.path.exists(folder):
        os.makedirs(folder)

roads_file = os.path.join(roads_data, "roads.gpkg")
speeds_file = os.path.join(base_folder, "speeds.gpkg")

In [43]:
def decode_openlr(openlr_string):
    """
    Decode OpenLR string to get the coordinates and other information.
    """
    # Decode the OpenLR string
    decoded = openlr.binary_decode(openlr_string)   
    # Extract the coordinates
    shp = LineString([Point(point.lat, point.lon) for point in decoded.points]    )
    return(shp)

try:
    in_speed = gpd.read_file(speeds_file)
except:    
    in_speed = pd.read_csv(input_speed_file, header=None)
    in_speed.columns = ['openlr'] + [f's_{x}' for x in range(0, len(in_speed.columns)-1)]
    in_speed['geometry'] = in_speed['openlr'].apply(decode_openlr)
    in_speed = gpd.GeoDataFrame(in_speed.loc[:,['openlr', 'geometry']], geometry='geometry', crs=4326)
    in_speed.to_file(speeds_file, driver='GPKG')


In [32]:
try: 
    in_roads = gpd.read_file(roads_file)
except:
    in_roads = osmnx.features_from_bbox(in_speed.total_bounds, {"highway":True})
    in_roads = gpd.GeoDataFrame(in_roads, geometry='geometry', crs=4326)
    in_roads.to_file(roads_file, driver='GPKG')

c:\wbg\Anaconda3\envs\mapbox\Lib\site-packages\osmnx\_overpass.py:267: UserWarning: This area is 35 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)
